In [1]:
import numpy as np
import pysal as ps
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def squaredDistance(point, segment):
    """Find the squared distance between a point and a segment
    
    Arguments
    =========
    
    point: tuple (x,y)
    
    segment: list of tuples [(x0,y0), (x1,y1)]
    
    Returns
    =======
    
    tuple: 2 elements
    
           distance squared between point and segment
    
           array(xb,yb): the nearest point on the segment
    
    """
    p0,p1 = [np.array(p) for p in segment]
    v = p1 - p0
    p = np.array(point)
    w = p - p0
    c1 = np.dot(w,v)
    if c1 <= 0.:
        # print 'before p0'
        return np.dot(w.T,w), p0
    c2 = np.dot(v,v)
    if c2 <= c1:
        dp1 = p - p1
        # print 'after p1'
        return np.dot(dp1.T,dp1), p1
    
    b = c1 / c2
    bv = np.dot(b,v)
    pb = p0 + bv
    d2 = p - pb
    
    return np.dot(d2,d2), pb
    
    

In [3]:
eb = ps.open(ps.examples.get_path("eberly_net.shp"))

In [4]:
chains = [ chain for chain in eb]

In [5]:
rt = ps.cg.Rtree()
SMALL = 0.01
node2segs = {}
for chain in chains[1:]:
    x0,y0,x1,y1 =  chain.bounding_box
    if (x0,y0) not in node2segs:
        node2segs[(x0,y0)] = []
    if (x1,y1) not in node2segs:
        node2segs[(x1,y1)] = []
    node2segs[(x0,y0)].append(chain)
    node2segs[(x1,y1)].append(chain)
    x0 -= SMALL
    y0 -= SMALL
    x1 += SMALL
    y1 += SMALL
    #print x0,y0,x1,y1
    r = ps.cg.Rect(x0,y0,x1,y1)
    #print r.area()
    rt.insert(chain, r)
   

In [6]:
kt = ps.cg.KDTree(node2segs.keys())

In [7]:
ep = ps.open(ps.examples.get_path("eberly_net_pts_offnetwork.shp"))

In [8]:
points = [ point for point in ep]

In [9]:
point

(9.025633038997064, 2.888497233975608)

In [10]:
dmin, node = kt.query(point,k=1)
kt.data[node]

array([ 8.,  4.])

In [13]:
p2s = {}
points.sort()

for point in points:
    #print point
    dmin, node = kt.query(point, k=1)
    node = tuple(kt.data[node])
    #print node
    closest = node2segs[node][0].vertices
    p2s[point] = (closest, point) # chain and snap point
    x0 = point[0] - dmin
    y0 = point[1] - dmin
    x1 = point[0] + dmin
    y1 = point[1] + dmin
    candidates = [ cand for cand in rt.intersection([x0,y0,x1,y1])]
    dmin += SMALL
    dmin2 = dmin * dmin
    #print "\n point ",point,"number of candidates: ",len(candidates)
    for candidate in candidates:
        dnc, p2b = squaredDistance(point, candidate.vertices)
        #print point, dnc, dmin2, p2b
        if dnc < dmin2:
            closest = candidate.vertices
            dmin2 = dnc
            p2s[point] = (closest, p2b)
    #print p2s[point]
   
    

    

In [14]:
ps = p2s.keys()
ps.sort()
for p in ps:
    print p,p2s[p]

(0.08503569102250197, 3.4291679735487035) ([(0.0, 4.0), (5.0, 4.0)], array([ 0.08503569,  4.        ]))
(0.10582418812288263, 2.8529527589939496) ([(0.0, 4.0), (5.0, 4.0)], array([ 0.10582419,  4.        ]))
(0.9221933143848327, 2.932646727847103) ([(0.0, 4.0), (5.0, 4.0)], array([ 0.92219331,  4.        ]))
(1.125205331703149, 4.8587675506025425) ([(1.0, 7.0), (0.0, 4.0)], array([ 0.3701508,  5.1104524]))
(1.2294216099258877, 4.528823930291667) ([(0.0, 4.0), (5.0, 4.0)], array([ 1.22942161,  4.        ]))
(1.5043534922448583, 3.1321032351822966) ([(0.0, 4.0), (5.0, 4.0)], array([ 1.50435349,  4.        ]))
(1.874678044967222, 4.0518996984423055) ([(0.0, 4.0), (5.0, 4.0)], array([ 1.87467804,  4.        ]))
(1.9947115329847633, 4.447281096736115) ([(3.0, 5.0), (2.0, 4.5)], array([ 2. ,  4.5]))
(2.0840557657189747, 5.8651611966484545) ([(3.0, 5.0), (2.0, 4.5)], array([ 2.61330909,  4.80665455]))
(2.291937682192724, 6.57834834523361) ([(1.0, 7.0), (0.0, 4.0)], array([ 1.,  7.]))
(2.48996